In [83]:
import cv2
import os
import random
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from skimage import feature
from sklearn.metrics import (f1_score, precision_score, recall_score, 
accuracy_score, confusion_matrix)
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import (RBF, Matern)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier

In [146]:
def criarListaImagens(listaCategorias, data_dir, img_size):
  lista = []
  for categoria in listaCategorias:
    path = os.path.join(data_dir, categoria)
    class_num = listaCategorias.index(categoria)
    for img in os.listdir(path):
      try:
        img_array = cv2.imread(os.path.join(path, img))
        new_array = cv2.resize(img_array, (img_size, img_size))
        lista.append([new_array, class_num])
      except Exception as e:
        print(e)
        pass
  return lista

#----------------------------------------------------------------------#

def criaListaTreinoTeste(lista):
  classes = []
  imagens = []
  for imagem in lista:
    imagens.append(imagem[0])
    classes.append(imagem[1])
  return imagens, classes

#----------------------------------------------------------------------#

def obterImagemPolar(image):
  img = image.astype(np.float32)
  value = np.sqrt(((img.shape[0]/2.0)**2.0)+((img.shape[1]/2.0)**2.0))
  polar_image = cv2.linearPolar(img,(img.shape[0]/2, img.shape[1]/2), value, cv2.WARP_FILL_OUTLIERS)
  return polar_image.astype(np.uint8)

#----------------------------------------------------------------------#

def obtemCanaisCoresRGB(listaOrigem):
    listaDestino = []
    for imagem in listaOrigem:
        b, g, r = cv2.split(imagem)
        gray = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
        aux = []
        aux.append(obterImagemPolar(b))
        aux.append(obterImagemPolar(g))
        aux.append(obterImagemPolar(r))
        aux.append(obterImagemPolar(gray))
        listaDestino.append(aux)

    return listaDestino

#----------------------------------------------------------------------#

def obterCanaisCores(listaOrigem, cvt):
    listaDestino = []
    for imagem in listaOrigem:    
        i = cv2.cvtColor(imagem, cvt)
        c1, c2, c3 = cv2.split(i)
        aux = []
        aux.append(obterImagemPolar(c1))
        aux.append(obterImagemPolar(c2))
        aux.append(obterImagemPolar(c3))
        aux.append(cv2.cvtColor(i, cv2.COLOR_BGR2GRAY))
        listaDestino.append(aux)

    return listaDestino

#----------------------------------------------------------------------#

class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
        self.numPoints = numPoints
        self.radius = radius
    def describe(self, image, eps=1e-7):
        lbp = feature.local_binary_pattern(image, self.numPoints,
            self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(),
            bins=np.arange(0, self.numPoints + 3),
            range=(0, self.numPoints + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)
        return hist

#----------------------------------------------------------------------#

def obtemHistogramaCanais(listaImagens, lbp):
  canal_1 = []
  canal_2 = []
  canal_3 = []
  canal_cinza = []
  for imagem in listaImagens:
    canal_1.append(lbp.describe(imagem[0]))
    canal_2.append(lbp.describe(imagem[1]))
    canal_3.append(lbp.describe(imagem[2]))
    canal_cinza.append(lbp.describe(imagem[3]))
  return canal_1, canal_2, canal_3, canal_cinza

#----------------------------------------------------------------------#

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]


classifiers = [
    KNeighborsClassifier(3),
    SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
        decision_function_shape='ovo', degree=3, gamma='scale', kernel='linear',
        max_iter=-1, probability=False, random_state=109, shrinking=True, tol=0.001,
        verbose=False),
    SVC(C=10000, cache_size=200, class_weight=None, coef0=0.0,
        decision_function_shape='ovo', degree=3, gamma=1, kernel='rbf', max_iter=-1,
        probability=False, random_state=109, shrinking=True, tol=0.001,verbose=False),    
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(class_weight=None, criterion='entropy',
        max_depth=3, max_features=None, max_leaf_nodes=None,
        min_impurity_decrease=0.0, min_impurity_split=None,
        min_samples_leaf=50, min_samples_split=2,
        min_weight_fraction_leaf=0.0,
        random_state=None, splitter='best'),
    RandomForestClassifier (bootstrap=False, class_weight=None,
        criterion='gini', max_depth=50, max_features='sqrt',
        max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None,
        min_samples_leaf=1, min_samples_split=2,
        min_weight_fraction_leaf=0.0, n_estimators=550,
        n_jobs=None, oob_score=False, random_state=None,
        verbose=0, warm_start=False),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()
]

#----------------------------------------------------------------------#

def xgboost(listaTreino, listaTeste, classesTreino, classesTeste):
    m = XGBClassifier(colsample_bytree= 0.6, 
        gamma= 5, learning_rate= 0.15, max_depth= 5, 
        min_child_weight= 1, n_estimators= 100, subsample=0.8, eta= 0.3 ).fit(listaTreino, classesTreino)
    preds = m.predict(listaTeste)
    y_pred = m.predict(listaTeste)     
    acc = accuracy_score(classesTeste, y_pred)
    f1  = f1_score(classesTeste, y_pred, average='weighted')
    cm  = confusion_matrix(classesTeste, y_pred)
    ps  = precision_score(classesTeste, y_pred, average='weighted')
    rs  = recall_score(classesTeste, y_pred, average='weighted')
    return acc, f1, cm, ps, rs

#----------------------------------------------------------------------#

def utilizarClassificadores(listaTreino, classesTreino, listaTeste, classesTeste, namesClassifiers, classifiers):
    listaAccuracy = {}
    listaCM = {}
    listaF1Score = {}
    listaPS = {}
    listaRS = {}
    for i, c in enumerate(classifiers):
        m = c.fit(listaTreino, classesTreino)
        y_pred = m.predict(listaTeste)         
        listaAccuracy[namesClassifiers[i]] = accuracy_score(classesTeste, y_pred)
        listaF1Score[namesClassifiers[i]]  = f1_score(classesTeste, y_pred, average='weighted')
        listaCM[namesClassifiers[i]]       = confusion_matrix(classesTeste, y_pred)
        listaPS[namesClassifiers[i]]       = precision_score(classesTeste, y_pred, average='weighted')
        listaRS[namesClassifiers[i]]       = recall_score(classesTeste, y_pred, average='weighted')
    listaAccuracy["XGBoost"], listaF1Score["XGBoost"], listaCM["XGBoost"], listaPS["XGBoost"], listaRS["XGBoost"] = xgboost(np.asarray(listaTreino), np.asarray(listaTeste), np.asarray(classesTreino), np.asarray(classesTeste))
    return listaAccuracy , listaF1Score, listaCM, listaPS, listaRS

In [147]:
data_dir = r"C:\Users\Rosana\Documents\Mestrado\DataSets\RIM_ONE_v2"
categorias = ["glaucoma","normal"]
img_size = 200
listaImagens = criarListaImagens(categorias, data_dir, img_size)
random.shuffle(listaImagens)
random.shuffle(listaImagens)

In [148]:
imagens, classes = criaListaTreinoTeste(listaImagens)
imagens_treino, imagens_teste, classes_treino, classes_teste = train_test_split(imagens, classes, test_size=0.33, random_state=42)

In [149]:
normal = 0
glaucoma = 0
for i in classes_treino:
    if (i == 0):
        glaucoma += 1
    else:
        normal += 1
        
print(normal)
print(glaucoma)

163
141


In [150]:
normal = 0
glaucoma = 0
for i in classes_teste:
    if (i == 0):
        glaucoma += 1
    else:
        normal += 1
        
print(normal)
print(glaucoma)

92
59


In [151]:
listaRgbTreino = obtemCanaisCoresRGB(imagens_treino)
listaHsvTreino = obterCanaisCores(imagens_treino, cv2.COLOR_BGR2HSV)
listaLabTreino = obterCanaisCores(imagens_treino, cv2.COLOR_BGR2LAB)
listaLuvTreino = obterCanaisCores(imagens_treino, cv2.COLOR_BGR2LUV)

listaRgbTeste = obtemCanaisCoresRGB(imagens_teste)
listaHsvTeste = obterCanaisCores(imagens_teste, cv2.COLOR_BGR2HSV)
listaLabTeste = obterCanaisCores(imagens_teste, cv2.COLOR_BGR2LAB)
listaLuvTeste = obterCanaisCores(imagens_teste, cv2.COLOR_BGR2LUV)

In [152]:
def classificarImagens(pontos, raio):
    _lbp = LocalBinaryPatterns(pontos, raio)

    lista_treino_rgb_b, lista_treino_rgb_g, lista_treino_rgb_r, lista_treino_rgb_gray = obtemHistogramaCanais(listaRgbTreino, _lbp)
    lista_treino_hsv_h, lista_treino_hsv_s, lista_treino_hsv_v, _ = obtemHistogramaCanais(listaHsvTreino, _lbp)
    lista_treino_lab_l, lista_treino_lab_a, lista_treino_lab_b, _ = obtemHistogramaCanais(listaLabTreino, _lbp)
    lista_treino_luv_l, lista_treino_luv_u, lista_treino_luv_v, _ = obtemHistogramaCanais(listaLuvTreino, _lbp)

    lista_teste_rgb_b, lista_teste_rgb_g, lista_teste_rgb_r, lista_teste_rgb_gray = obtemHistogramaCanais(listaRgbTeste, _lbp)
    lista_teste_hsv_h, lista_teste_hsv_s, lista_teste_hsv_v, _ = obtemHistogramaCanais(listaHsvTeste, _lbp)
    lista_teste_lab_l, lista_teste_lab_a, lista_teste_lab_b, _ = obtemHistogramaCanais(listaLabTeste, _lbp)
    lista_teste_luv_l, lista_teste_luv_u, lista_teste_luv_v, _ = obtemHistogramaCanais(listaLuvTeste, _lbp)

    linha = []
    listaF1Score = []
    listaCM = [] 
    conf_matrix = []
    listaPS = []
    listaRS = []

    #('------------------ RGB ----------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_rgb_gray, classes_treino, lista_teste_rgb_gray, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)
    #('------------------ RGB - R ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_rgb_r, classes_treino, lista_teste_rgb_r, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)
    #('------------------ RGB - G ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_rgb_g, classes_treino, lista_teste_rgb_g, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)
    #('------------------ RGB - B ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_rgb_b, classes_treino, lista_teste_rgb_b, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)

    #('------------------ HSV - H ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_hsv_h, classes_treino, lista_teste_hsv_h, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)
    #('------------------ HSV - S ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_hsv_s, classes_treino, lista_teste_hsv_s, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)
    #('------------------ HSV - V ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_hsv_v, classes_treino, lista_teste_hsv_v, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)
    
    
    #('------------------ LAB - L ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_lab_l, classes_treino, lista_teste_lab_l, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)
    #('------------------ LAB - A ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_lab_a, classes_treino, lista_teste_lab_a, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)
    #('------------------ LAB - B ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_lab_b, classes_treino, lista_teste_lab_b, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)
    
    #('------------------ LUV - L ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_luv_l, classes_treino, lista_teste_luv_l, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)    
    
    #('------------------ LUV - U ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_luv_u, classes_treino, lista_teste_luv_u, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)    
    
    #('------------------ LUV - V ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_luv_v, classes_treino, lista_teste_luv_v, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)    
        
    return linha, listaF1Score, listaCM, listaPS, listaRS

In [153]:
def obterResultados (pontos, raio):
    linhas, listaF1Score, listaCM, listaPS, listaRS = classificarImagens(pontos, raio)
    print('------ %s pontos - %s raio ------' % (pontos, raio))
    print('--- Acuracia ---')
    display(pd.DataFrame(linhas))
    print('--- F1 ---')
    display(pd.DataFrame(listaF1Score))
    print('--- Precision Score ---')
    display(pd.DataFrame(listaPS))
    print('--- Recall Score ---')
    display(pd.DataFrame(listaRS))
    print('--- Matriz confusao ---')
    display(pd.DataFrame(listaCM))
    print('---------------------------------')
    

In [ ]:
obterResultados(4,1)
obterResultados(8,1)
obterResultados(8,2)
obterResultados(12,2)
obterResultados(16,2)
obterResultados(16,3)
obterResultados(24,3)
obterResultados(24,8)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined 

------ 4 pontos - 1 raio ------
--- Acuracia ---


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


,Nearest Neighbors,Linear SVM,RBF SVM,Gaussian Process,Decision Tree,Random Forest,Neural Net,AdaBoost,Naive Bayes,QDA,XGBoost
0,0.801325,0.847682,0.874172,0.834437,0.781457,0.827815,0.609272,0.807947,0.728477,0.794702,0.841060
1,0.728477,0.814570,0.801325,0.807947,0.754967,0.794702,0.609272,0.728477,0.701987,0.688742,0.794702
2,0.807947,0.847682,0.874172,0.854305,0.814570,0.827815,0.609272,0.801325,0.735099,0.781457,0.847682
3,0.801325,0.854305,0.880795,0.847682,0.735099,0.821192,0.609272,0.841060,0.748344,0.761589,0.834437
4,0.735099,0.735099,0.741722,0.741722,0.688742,0.715232,0.609272,0.715232,0.708609,0.642384,0.721854
5,0.708609,0.788079,0.794702,0.741722,0.655629,0.735099,0.609272,0.741722,0.728477,0.781457,0.754967
6,0.728477,0.814570,0.801325,0.807947,0.754967,0.794702,0.609272,0.728477,0.701987,0.682119,0.794702
7,0.801325,0.841060,0.867550,0.834437,0.774834,0.854305,0.609272,0.821192,0.748344,0.774834,0.821192
8,0.735099,0.781457,0.807947,0.774834,0.701987,0.682119,0.615894,0.748344,0.761589,0.768212,0.754967
9,0.761589,0.841060,0.854305,0.821192,0.821192,0.768212,0.609272,0.761589,0.794702,0.715232,0.801325


--- F1 ---


...

--- Precision Score ---


,Nearest Neighbors,Linear SVM,RBF SVM,Gaussian Process,Decision Tree,Random Forest,Neural Net,AdaBoost,Naive Bayes,QDA,XGBoost
0,0.705599,0.806681,0.792901,0.819909,0.757581,0.820238,0.371212,0.714401,0.779545,0.763413,0.779900
1,0.671223,0.780322,0.792901,0.826656,0.654657,0.729339,0.371212,0.758858,0.724157,0.731058,0.758953
2,0.682119,0.820705,0.801325,0.826656,0.767496,0.836468,0.371212,0.780841,0.806392,0.774298,0.797024
3,0.649635,0.733270,0.725291,0.628660,0.689629,0.720418,0.371212,0.738228,0.731193,0.681515,0.757071
4,0.735099,0.808433,0.779900,0.833906,0.715837,0.772456,0.371212,0.705599,0.731680,0.678698,0.731680
5,0.712680,0.724560,0.735099,0.772853,0.573290,0.750038,0.371212,0.704501,0.717500,0.719475,0.767553
6,0.671223,0.780322,0.792901,0.826656,0.654657,0.729339,0.371212,0.758858,0.724157,0.724157,0.758953
7,0.661249,0.779358,0.807417,0.813662,0.655813,0.779900,0.371212,0.744433,0.765564,0.706087,0.765698
8,0.754265,0.860459,0.833993,0.833462,0.797978,0.813662,0.371212,0.759402,0.813215,0.746980,0.833426
9,0.853557,0.846853,0.814570,0.860459,0.765841,0.841060,0.371212,0.766509,0.840335,0.784922,0.833993


--- Recall Score ---


,Nearest Neighbors,Linear SVM,RBF SVM,Gaussian Process,Decision Tree,Random Forest,Neural Net,AdaBoost,Naive Bayes,QDA,XGBoost
0,0.708609,0.807947,0.794702,0.821192,0.754967,0.821192,0.609272,0.715232,0.781457,0.748344,0.781457
1,0.675497,0.781457,0.794702,0.827815,0.655629,0.728477,0.609272,0.761589,0.728477,0.735099,0.761589
2,0.682119,0.821192,0.801325,0.827815,0.761589,0.834437,0.609272,0.781457,0.807947,0.615894,0.794702
3,0.642384,0.708609,0.728477,0.622517,0.675497,0.715232,0.609272,0.741722,0.735099,0.668874,0.748344
4,0.735099,0.807947,0.781457,0.834437,0.701987,0.774834,0.609272,0.708609,0.735099,0.682119,0.735099
5,0.708609,0.728477,0.735099,0.774834,0.596026,0.748344,0.609272,0.708609,0.721854,0.721854,0.768212
6,0.675497,0.781457,0.794702,0.827815,0.655629,0.728477,0.609272,0.761589,0.728477,0.728477,0.761589
7,0.662252,0.781457,0.807947,0.814570,0.629139,0.781457,0.609272,0.741722,0.768212,0.688742,0.768212
8,0.754967,0.860927,0.834437,0.834437,0.768212,0.814570,0.609272,0.761589,0.814570,0.735099,0.834437
9,0.854305,0.847682,0.814570,0.860927,0.768212,0.841060,0.609272,0.768212,0.841060,0.768212,0.834437


--- Matriz confusao ---
